In [1]:
import pandas as pd
import numpy as np
import os

# Read Excel file and remove development group rows
file_path = "src/raw_data/undesa_pd_2024_ims_stock_by_sex_and_destination.xlsx"
df_raw = pd.read_excel(file_path, sheet_name="Table 1", header=9, engine='openpyxl')

df_clean = df_raw.drop(df_raw.index[3:27])
df_clean = df_clean.reset_index(drop=True)

In [2]:
# Create dynamic continent/region mapping from Excel structure
region_col = df_clean.columns[1]
current_continent = None
current_region = None
country_mapping = {}

continent_patterns = {
    "AFRICA": "AFRICA",
    "Sub-Saharan Africa": "AFRICA", 
    "Northern Africa and Western Asia": "AFRICA",
    "ASIA": "ASIA",
    "Central and Southern Asia": "ASIA",
    "Eastern and South-Eastern Asia": "ASIA", 
    "EUROPE": "EUROPE",
    "Europe and Northern America": "EUROPE",
    "LATIN AMERICA AND THE CARIBBEAN": "LATIN AMERICA AND THE CARIBBEAN",
    "NORTHERN AMERICA": "NORTHERN AMERICA", 
    "OCEANIA": "OCEANIA"
}

region_indicators = [
    "Eastern", "Western", "Northern", "Southern", "Central", "Middle",
    "Caribbean", "Melanesia", "Polynesia", "Micronesia", 
    "Australia and New Zealand", "Pacific"
]

for i in range(len(df_clean)):
    row = df_clean.iloc[i]
    name = str(row[region_col]) if pd.notna(row[region_col]) else ""
    
    if name == "" or name == "nan":
        continue
    
    if name in continent_patterns:
        current_continent = continent_patterns[name]
        current_region = None
        continue
    
    is_region = any(indicator in name for indicator in region_indicators)
    
    if is_region and current_continent:
        current_region = name
        continue
    
    if current_continent and not is_region and name not in continent_patterns:
        if current_region:
            country_mapping[name] = (current_continent, current_region)
        else:
            country_mapping[name] = (current_continent, current_continent)

In [3]:
# Create final dataframe with countries only
country_data_rows = []
region_col = df_clean.columns[1]

for i in range(len(df_clean)):
    row = df_clean.iloc[i]
    country_name = str(row[region_col]) if pd.notna(row[region_col]) else ""
    
    if country_name in country_mapping:
        country_data_rows.append(row)

df_countries = pd.DataFrame(country_data_rows)
df_countries = df_countries.reset_index(drop=True)

final_df = pd.DataFrame()
final_df['country'] = df_countries[region_col]

continent_list = []
region_list = []

for country in final_df['country']:
    if country in country_mapping:
        continent, region = country_mapping[country]
        continent_list.append(continent)
        region_list.append(region)
    else:
        continent_list.append('Unknown')
        region_list.append('Unknown')

final_df['continent'] = continent_list
final_df['region'] = region_list

# Add year data starting from column 5
years = [1990, 1995, 2000, 2005, 2010, 2015, 2020, 2024]
col_index = 5

for year in years:
    if col_index + 2 < len(df_countries.columns):
        both_col = df_countries.columns[col_index]
        male_col = df_countries.columns[col_index + 1] 
        female_col = df_countries.columns[col_index + 2]
        
        final_df[str(year)] = pd.to_numeric(df_countries[both_col], errors='coerce').fillna(0).astype(int)
        final_df[f"{year}_male"] = pd.to_numeric(df_countries[male_col], errors='coerce').fillna(0).astype(int)
        final_df[f"{year}_female"] = pd.to_numeric(df_countries[female_col], errors='coerce').fillna(0).astype(int)
        
        col_index += 3

In [4]:
# Export to CSV
output_path = 'src/output/migration_data_processed.csv'
final_df.to_csv(output_path, index=False, quoting=1)